# Bab 11: Training Deep Neural Networks (Melatih Jaringan Saraf Tiruan Dalam)

### 1. Pendahuluan

Bab 11 memperkenalkan dasar-dasar Jaringan Saraf Tiruan (JST) dan Keras. Bab 11 ini akan menyelami tantangan utama dalam melatih *Deep Neural Networks* (DNN) – JST dengan banyak lapisan tersembunyi – dan bagaimana mengatasinya. DNN memiliki potensi yang sangat besar, tetapi dapat menghadapi berbagai masalah.

Masalah utama yang dibahas di bab ini meliputi:
* **Vanishing/Exploding Gradients:** Gradien menjadi sangat kecil atau sangat besar, menghambat atau mengacaukan pelatihan.
* **Non-convergence of Gradient Descent:** *Gradient Descent* dapat terjebak dalam *local optima* atau membutuhkan waktu sangat lama untuk konvergen.
* **Overfitting:** DNN dengan banyak parameter sangat rentan terhadap *overfitting*.
* **Lambatnya Pelatihan:** DNN membutuhkan waktu lama untuk dilatih.

Bab ini akan menyajikan berbagai teknik untuk mengatasi masalah-masalah ini, memungkinkan pelatihan model yang lebih dalam dan lebih efisien.

### 2. Masalah Vanishing/Exploding Gradients (The Vanishing/Exploding Gradients Problem)

Fenomena ini terjadi selama *backpropagation*:
* **Vanishing Gradients:** Gradien seringkali menjadi semakin kecil saat algoritma bergerak mundur melalui lapisan-lapisan, menuju lapisan input. Ini berarti *update* bobot pada lapisan-lapisan awal menjadi sangat kecil, dan pelatihan berhenti belajar secara efektif. Masalah ini sangat umum dengan fungsi aktivasi Sigmoid dan Tanh.
* **Exploding Gradients:** Ini adalah masalah sebaliknya, di mana gradien dapat menjadi sangat besar, menyebabkan *update* bobot menjadi terlalu besar dan algoritma divergen. Ini lebih sering terjadi pada *recurrent neural networks* (RNNs).

Masalah ini adalah salah satu alasan mengapa *Deep Learning* sempat mengalami stagnasi hingga pertengahan 2000-an.

### 3. Mengatasi Masalah Vanishing/Exploding Gradients (Tackling the Vanishing/Exploding Gradients Problem)

Beberapa teknik telah dikembangkan untuk mengatasi masalah ini:

#### a. Glorot Initialization (Xavier Initialization)
Ini adalah strategi untuk inisialisasi bobot (weights) model. Ini memastikan bahwa varians output dari setiap lapisan (dan varians gradien pada *backward pass*) tetap kurang lebih sama di seluruh lapisan. Ini membantu mencegah sinyal (baik data maupun gradien) mati atau meledak terlalu dini.
* Untuk fungsi aktivasi ReLU (dan variasinya) atau *linear*, Glorot initialization menginisialisasi bobot dari distribusi Gaussian dengan mean 0 dan varians $\sigma^2 = \frac{2}{n_{in} + n_{out}}$ atau distribusi seragam.
* `keras.layers.Dense` secara *default* menggunakan Glorot initialization dengan distribusi seragam.

#### b. He Initialization
Ini adalah inisialisasi yang disarankan untuk fungsi aktivasi ReLU dan variasinya. Mirip dengan Glorot, tetapi variansnya $\sigma^2 = \frac{2}{n_{in}}$.
* Anda dapat menggunakan `kernel_initializer="he_normal"` atau `kernel_initializer="he_uniform"` di lapisan Keras.

#### c. Non-saturating Activation Functions (Fungsi Aktivasi Non-saturasi)
Menggunakan fungsi aktivasi yang tidak "jenuh" pada input besar atau kecil (di mana gradien menjadi sangat datar):
* **ReLU (Rectified Linear Unit):** `f(z) = max(0, z)`. Cepat dihitung, tidak ada masalah *vanishing gradient* untuk input positif.
    * **Masalah *Dying ReLUs*:** Neuron bisa "mati" jika outputnya selalu negatif, menyebabkan gradiennya selalu nol dan neuron berhenti belajar.
* **Leaky ReLU:** `f(z) = max(az, z)` di mana $a$ adalah konstanta kecil (misalnya, 0.01). Ini memastikan neuron tidak pernah mati total.
* **PReLU (Parametric Leaky ReLU):** Mirip dengan Leaky ReLU, tetapi $a$ adalah parameter yang dipelajari selama pelatihan.
* **ELU (Exponential Linear Unit):** `f(z) = z` jika $z \ge 0$, dan `a(exp(z) - 1)` jika $z < 0$. Ini mengungguli semua fungsi aktivasi ReLU lainnya.
* **SELU (Scaled Exponential Linear Unit):** Jika dibangun dengan benar (lapisan `Dense` saja, inisialisasi He, penskalaan input standar), jaringan yang menggunakan SELU dapat melakukan normalisasi diri, membuat jaringan sangat dalam tanpa masalah *vanishing/exploding gradients*.

#### d. Batch Normalization (Normalisasi Batch)
Batch Normalization (BN) adalah teknik yang sangat efektif dan terpopuler untuk mengatasi masalah *vanishing/exploding gradients*. Ini juga bertindak sebagai regularisasi.
* Ini menambahkan operasi Normalisasi Batch di setiap lapisan (atau setelah fungsi aktivasi).
* Ia menormalisasi input lapisan dengan mengurangi *mean* dan membagi dengan *standard deviation* *batch* saat ini, kemudian menskalakan dan menggesernya menggunakan dua set parameter yang dipelajari per lapisan (`gamma` dan `beta`).
* Ini mengurangi masalah *Internal Covariate Shift* (perubahan distribusi input setiap lapisan selama pelatihan).
* `keras.layers.BatchNormalization()`: Dapat ditambahkan sebagai lapisan.

#### e. Gradient Clipping (Pemotongan Gradien)
Untuk mengatasi *exploding gradients*, terutama di RNN, Anda dapat memotong gradien agar tidak melebihi ambang batas tertentu. Ini dapat dilakukan dengan mengatur `clipvalue` atau `clipnorm` di *optimizer* Keras.

### 4. Mentransfer Pembelajaran (Transfer Learning)

*Transfer Learning* adalah teknik yang sangat kuat di mana Anda menggunakan sebagian dari model terlatih sebelumnya (biasanya model yang sangat dalam dan dilatih pada dataset besar) dan menggunakannya kembali sebagai dasar untuk tugas baru. Ini mengurangi waktu dan data yang dibutuhkan untuk melatih model baru.

* **Pembekuan Lapisan:** Lapisan-lapisan awal dari model yang sudah dilatih biasanya "dibekukan" (tidak diperbarui selama pelatihan) karena mereka telah mempelajari fitur tingkat rendah yang generik.
* **Lapisan Output Baru:** Lapisan output model lama diganti dengan lapisan output baru yang cocok untuk tugas baru.
* **Penyesuaian (Fine-tuning):** Setelah beberapa *epoch* dengan lapisan yang dibekukan, Anda dapat mencoba "mencairkan" beberapa lapisan teratas (lapisan yang lebih dekat ke output) dan melatih kembali seluruh model dengan *learning rate* yang sangat rendah.

### 5. Optimizer Lebih Cepat (Faster Optimizers)

*Gradient Descent* standar seringkali terlalu lambat. Banyak *optimizer* yang lebih canggih telah dikembangkan:

#### a. Momentum Optimizer
Mempercepat *Gradient Descent* dengan menambahkan "momentum" ke *gradient*. Ia tidak hanya menggunakan *gradient* saat ini tetapi juga *gradient* dari langkah-langkah sebelumnya, mirip bola yang menggelinding menuruni bukit.
* `keras.optimizers.SGD(momentum=...)`

#### b. Nesterov Accelerated Gradient (NAG)
Versi *momentum* yang lebih baik yang mengukur *gradient* tidak pada posisi saat ini, tetapi sedikit di depan ke arah momentum. Ini biasanya konvergen lebih cepat daripada Momentum biasa.
* `keras.optimizers.SGD(nesterov=True)`

#### c. AdaGrad
Menyesuaikan *learning rate* untuk setiap parameter secara individual. Ini mengurangi *learning rate* untuk parameter yang sering diperbarui dan meningkatkan untuk parameter yang jarang diperbarui. Cocok untuk masalah *sparse*.
* `keras.optimizers.Adagrad()`
* **Kekurangan:** *Learning rate* dapat berkurang terlalu cepat, menghentikan pembelajaran.

#### d. RMSProp
Mengatasi masalah AdaGrad dengan hanya mengumpulkan gradien dari iterasi terbaru, bukan semua gradien masa lalu.
* `keras.optimizers.RMSprop()`

#### e. Adam (Adaptive Moment Estimation)
Menggabungkan ide *Momentum* dan RMSProp. Ia melacak rata-rata eksponensial dari *gradient* kuadrat dan rata-rata eksponensial dari gradien itu sendiri. Ini seringkali merupakan *optimizer* default yang baik.
* `keras.optimizers.Adam()`

#### f. Adamax
Varian Adam yang didasarkan pada norma tak terbatas (L-infinity norm). Mungkin lebih baik dalam beberapa kasus, terutama dengan fitur *sparse*.

#### g. Nadam
Adam + Nesterov momentum.

**Memilih Optimizer:** Adam, RMSProp, dan Nesterov Accelerated Gradient (NAG) adalah pilihan yang baik. Adam seringkali merupakan titik awal yang baik.

### 6. Menghindari Overfitting Melalui Regularisasi (Avoiding Overfitting Through Regularization)

DNN memiliki jutaan parameter dan sangat rentan terhadap *overfitting*.

#### a. L1 dan L2 Regularization (Regularisasi L1 dan L2)
Menambahkan penalti pada fungsi biaya berdasarkan ukuran bobot model.
* **L1 (Lasso):** Menambahkan $\ell_1$ norm dari bobot. Cenderung mendorong bobot ke nol (seleksi fitur).
* **L2 (Ridge/Weight Decay):** Menambahkan $\ell_2$ norm kuadrat dari bobot. Mendorong bobot menjadi kecil tetapi tidak nol.
* Dapat ditambahkan ke lapisan Keras menggunakan argumen `kernel_regularizer` dan `bias_regularizer`.

#### b. Dropout
Dropout adalah teknik regularisasi yang sangat populer dan efektif.
* Pada setiap langkah pelatihan, setiap neuron di lapisan tertentu (kecuali lapisan output) memiliki probabilitas $p$ untuk "dihilangkan" (outputnya disetel ke nol sementara).
* Ini memaksa neuron untuk belajar untuk tidak terlalu bergantung pada neuron tetangga tertentu, membuat model lebih kuat dan kurang rentan *overfitting*.
* `keras.layers.Dropout()`: Dapat ditambahkan sebagai lapisan.
* **Catatan:** Dropout hanya aktif selama pelatihan; selama inferensi, neuron tidak dihilangkan.

#### c. Alpha Dropout
Varian Dropout yang bekerja dengan SELU. Ini menjaga properti normalisasi diri SELU.

#### d. Monte Carlo (MC) Dropout
Melakukan prediksi beberapa kali dengan Dropout aktif dan mengambil rata-rata, yang dapat memberikan estimasi kepercayaan dan meningkatkan kinerja.

#### e. Max-Norm Regularization
Membatasi $\ell_2$ norm dari vektor bobot input neuron. Setelah setiap langkah pelatihan, jika $\ell_2$ norm bobot melebihi batas, bobot diskalakan kembali. Ini juga membantu mengatasi *exploding gradients*.

### 7. Ringkasan Praktis (Practical Guidelines)

Bab ini diakhiri dengan ringkasan praktis tentang *hyperparameter* JST:

| Hyperparameter        | Default Baik (Good Default) | Perubahan untuk Underfitting | Perubahan untuk Overfitting |
| :-------------------- | :-------------------------- | :--------------------------- | :-------------------------- |
| Inisialisasi Bobot    | He Initialization (ReLU/ELU/SELU) | -                            | -                           |
| Fungsi Aktivasi       | ReLU (terutama di hidden layers), Softmax (output klasifikasi), Sigmoid (output biner) | -                            | -                           |
| Optimizer             | Adam, Nadam, RMSProp        | Naikkan *learning rate* | Turunkan *learning rate* |
| Learning Rate         | ~1e-3 hingga 3e-3           | Naikkan                       | Turunkan                    |
| Ukuran Batch          | 32                          | -                            | Turunkan                    |
| Jumlah Layer Tersembunyi | 2-5                         | Tambah                       | Kurangi                     |
| Neuron per Layer      | 10-100                      | Tambah                       | Kurangi                     |
| Regularisasi L1/L2    | Tidak ada secara default, bisa ditambahkan | Kurangi                       | Tambah                      |
| Dropout Rate          | Tidak ada secara default, 0.2-0.5 jika ditambahkan | Kurangi                       | Tambah                      |
| Batch Normalization   | Sangat disarankan           | -                            | -                           |
| Early Stopping        | Sangat disarankan           | -                            | -                           |

### 8. Kesimpulan 

Bab 11 sangat penting untuk pelatihan DNN yang efektif. Ini mengidentifikasi dan memberikan solusi untuk masalah *vanishing/exploding gradients* (inisialisasi yang lebih baik, fungsi aktivasi non-saturasi, Batch Normalization, Gradient Clipping), masalah *overfitting* (regularisasi L1/L2, Dropout), dan meningkatkan kecepatan pelatihan (*faster optimizers*). Pengenalan teknik *transfer learning* juga membuka jalan untuk memanfaatkan model yang sudah dilatih pada tugas-tugas baru. Bab ini melengkapi pembaca dengan "kotak peralatan" yang diperlukan untuk membangun dan melatih JST yang dalam dan tangguh.


## 1. Setup

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd # For plotting history

## 2. Vanishing/Exploding Gradients Problem

### Glorot and He Initialization

In [ ]:
# Glorot initialization is the default for Keras Dense layers (kernel_initializer='glorot_uniform')
# He initialization can be specified using 'he_normal' or 'he_uniform'
# Example:
keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal")

### Leaky ReLU

In [ ]:
# Creating a Leaky ReLU activation layer
leaky_relu_activation = keras.layers.LeakyReLU(alpha=0.2)

# Example model with Leaky ReLU
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer="he_normal"),
    leaky_relu_activation,
    keras.layers.Dense(100, kernel_initializer="he_normal"),
    leaky_relu_activation,
    keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

### PReLU (Parametric ReLU)

In [ ]:
# PReLU is implemented as a Keras layer
keras.layers.PReLU()

### ELU (Exponential Linear Unit)

In [ ]:
# ELU is a Keras activation function
# Example:
keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal")

### SELU (Scaled Exponential Linear Unit)

In [ ]:
# SELU is a Keras activation function
# Example:
keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal")
# For SELU to work properly, inputs must be standardized (mean 0, std 1)
# and network must be purely sequential (no skip connections, etc.)

## 3. Batch Normalization

In [ ]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(), # BN layer after Flatten (input layer)
    keras.layers.Dense(300, activation="relu"),
    keras.layers.BatchNormalization(), # BN layer after hidden layer
    keras.layers.Dense(100, activation="relu"),
    keras.layers.BatchNormalization(), # BN layer after hidden layer
    keras.layers.Dense(10, activation="softmax")
])

In [ ]:
# You can also add BN before activation function
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(100, kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(10, activation="softmax")
])

In [ ]:
# Compile and fit the model (assuming X_train, y_train, X_valid, y_valid are loaded)
model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

## 4. Gradient Clipping

In [ ]:
# Use an optimizer with gradient clipping
# Example: SGD with clipvalue
optimizer = keras.optimizers.SGD(clipvalue=1.0) # Clip gradients to max value 1.0
optimizer = keras.optimizers.SGD(clipnorm=1.0) # Clip gradients by norm
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

## 5. Reusing Pretrained Layers (Transfer Learning)

In [ ]:
# Load a pre-trained model (e.g., from Keras Applications)
# Note: This is an example, you might load a model like VGG16, ResNet, etc.
# For simplicity, let's pretend we have a pre-trained model.
# In a real scenario, you would load a model like:
# base_model = keras.applications.ResNet50(weights="imagenet", include_top=False)

# For demonstration, let's create a dummy base model for structure
base_model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(50, activation="relu")
])

# Create a new model on top of the base model
model = keras.models.Sequential([
    base_model,
    keras.layers.Dense(10, activation="softmax")
])

### Freezing the base model's layers

In [ ]:
base_model.trainable = False # Freeze the base model

In [ ]:
# Compile and train the new model with frozen layers
model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

### Unfreezing layers (fine-tuning)

In [ ]:
base_model.trainable = True # Unfreeze the base model
# It's important to recompile the model after unfreezing layers
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-4), # Use a very low learning rate
              metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

## 6. Faster Optimizers

In [ ]:
# Momentum Optimizer
optimizer = keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)

In [ ]:
# Nesterov Accelerated Gradient (NAG)
optimizer = keras.optimizers.SGD(learning_rate=0.001, momentum=0.9, nesterov=True)

In [ ]:
# AdaGrad Optimizer
optimizer = keras.optimizers.Adagrad(learning_rate=0.001)

In [ ]:
# RMSProp Optimizer
optimizer = keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9)

In [ ]:
# Adam Optimizer (often a good default)
optimizer = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)

In [ ]:
# Adamax Optimizer
optimizer = keras.optimizers.Adamax(learning_rate=0.001, beta_1=0.9, beta_2=0.999)

In [ ]:
# Nadam Optimizer
optimizer = keras.optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)

In [ ]:
# Example of compiling with an optimizer
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

## 7. Regularization

### L1 and L2 Regularization

In [ ]:
# L2 regularization on Dense layer
kernel_regularizer=keras.regularizers.l2(0.01)
bias_regularizer=keras.regularizers.l2(0.01)
activity_regularizer=keras.regularizers.l2(0.01)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="relu",
                       kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dense(100, activation="relu",
                       kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

### Dropout

In [ ]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dropout(rate=0.2), # Dropout layer after Flatten
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dropout(rate=0.2), # Dropout layer after hidden layer
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dropout(rate=0.2), # Dropout layer after hidden layer
    keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

### Alpha Dropout

In [ ]:
# Use with SELU activation and lecun_normal initializer
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.Dense(300, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.Dense(10, activation="softmax")
])

### Max-Norm Regularization

In [ ]:
# Implemented using a Keras constraint
from keras.constraints import max_norm
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="relu", kernel_constraint=max_norm(3)),
    keras.layers.Dense(100, activation="relu", kernel_constraint=max_norm(3)),
    keras.layers.Dense(10, activation="softmax")
])